In [ ]:
#!pip install pytrec-eval
#!pip install ipywidgets
#!pip install --upgrade pip

In [ ]:
print('hi')

In [ ]:
import open_search_connect 
client=open_search_connect.connect()


In [ ]:
from opensearchpy import OpenSearch

def search_opensearch(query, k=10):
    body = {
        "size": k,
        "query": {
            "match": {
                "text": {
                    "query": query
                }
            }
        }
    }
    res = client.search(index=INDEX_NAME, body=body)
  #  print(res["hits"]["hits"])
    out = [hit["_id"] for hit in res["hits"]["hits"]]
    return out


In [ ]:
search_opensearch("amber urine",20)

In [ ]:
# modified  return value for running  pytrec_eval 
from opensearchpy import OpenSearch

def search_opensearch_reranked(query, k):
    body = {
        "size": k,
        "query": {
            "match": {
                "text": {
                    "query": query,
                    "analyzer": "english"

                }
            }
        }
    }
    res = client.search(index=INDEX_NAME, body=body)
    out = [(hit["_source"]["doc_id"], hit["_source"]["text"]) for hit in res["hits"]["hits"]]
    return out


In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
# modified  return value for running  pytrec_eval 
from opensearchpy import OpenSearch

def hybrid_opensearch_reranked(query, k):
    query_vector = embedding_model.encode(query).tolist()
 
    body = {
        "size": k,
        "query": {
            "should"{
            "match": {
                "text": {
                    "query": query,
                    "analyzer": "english"

                }
            },
                                
             "knn": {
                            "text_vector": {
                                "vector": query_vector,
                                "k": size,
                                "boost": 3.0
                            }
                        }
                    },

        }
        }
    }
    res = client.search(index=INDEX_NAME, body=body)
#   print(res["hits"]["hits"])
   # out = [hit["_id"] for hit in res["hits"]["hits"]]
    out = [(hit["_source"]["doc_id"], hit["_source"]["text"]) for hit in res["hits"]["hits"]]
    #print("----")
    # for index,hit in enumerate(out):
    #     print(index+1,"--", hit[0],"----", hit[1][:100])
    return out


In [ ]:
single_docs = search_opensearch_reranked("amber urine",1)
print(single_docs)
#rerank_with_cross_encoder("amber urine",single_docs)

In [ ]:
from sentence_transformers import CrossEncoder
model = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")


In [ ]:
# query = "how tall is mount everest?"
# doc = "Mount Everest is 8848 meters high."

# score = model.predict([(query, doc)])
# print(score)  # e.g. [0.92]


In [ ]:
def rerank_with_cross_encoder(query, docs,rerank_count):
    pairs = [(query, text) for _, text in docs[:rerank_count]]
    scores = model.predict(pairs)

    reranked = sorted(
        zip(docs, scores),
        key=lambda x: x[1],
        reverse=True
    )

    return [(doc_id, text, score) for ((doc_id, text), score) in reranked]


In [ ]:
# query = " )what was the immediate impact of the success of the manhattan project? "
# result_docs = search_opensearch(query, k=20)


In [ ]:
#reranked = rerank_with_cross_encoder(query, result_docs)


In [ ]:
import ir_datasets
dataset = ir_datasets.load("msmarco-passage/train/split200-train")
queries = {q.query_id: q.text for q in dataset.queries_iter()}
qrels = dataset.qrels_dict()   # ✅ use this

In [ ]:
#Evaluates opensearch results against the rlevant documents in qrel_dict
import pytrec_eval

from tqdm import tqdm

run = {}

for qid, query_text in tqdm(list(queries.items())[:20]):  # limit for speed
    docs = search_opensearch(query_text, k=20)
    #print(docs)
    #    print(qid, query_text) 
    
    run[qid] = {doc_id: 1.0 for doc_id in docs}  # dummy score
    # for doc_id in reranked:
    #    print(doc_id[0])

evaluator = pytrec_eval.RelevanceEvaluator(
    qrels,
    {"map", "ndcg_cut_10", "recall_10", "P_10"}
)

results = evaluator.evaluate(run)

# average over queries
avg = {m: 0 for m in ["map", "ndcg_cut_10", "recall_10", "P_10"]}

for qid in results:
    for m in avg:
        avg[m] += results[qid][m]

for m in avg:
    avg[m] /= len(results)

print(avg)


In [ ]:
{'map': 0.021794871794871794, 'ndcg_cut_10': 0.028711770538226204, 'recall_10': 0.06666666666666667, 'P_10': 0.006666666666666667}


In [ ]:
#Calculate "run" for evaluations with re-ranking
from tqdm import tqdm
run = {}

for qid, query_text in tqdm(list(queries.items())[:500]):  # limit for speed
    if(qid in qrels):
        docs = search_opensearch_reranked(query_text,100)
        reranked_docs = rerank_with_cross_encoder(query_text, docs,100)
        
        run[qid] = {
        doc_id: float(score)
        for doc_id, _, score in reranked_docs[:10]
        }


In [ ]:
import pytrec_eval

evaluator = pytrec_eval.RelevanceEvaluator(qrels,{"map", "ndcg_cut_10", "recall_10", "P_10","recip_rank"})
results = evaluator.evaluate(run)

# average over queries
avg = {m: 0 for m in ["map", "ndcg_cut_10", "recall_10", "P_10","recip_rank"]}

for qid in results:
    for m in avg:
        avg[m] += results[qid][m]

for m in avg:
    avg[m] /= len(results)
    #print(avg)


print(avg)


In [ ]:
len(results)

In [ ]:
print(results)